# Usando Transformers con cuantificación de 8 bits

Para utilizar la cuantificación de 8 bits en modelos de Transformers, puedes usar el argumento `load_in_8bit=True` en el método `from_pretrained`. Esto te permite cargar un modelo reduciendo aproximadamente a la mitad los requisitos de memoria. Este método es compatible con modelos que admiten la carga con la biblioteca Accelerate y que contienen capas `torch.nn.Linear`.

NOTA: Necesita tener instalado accelerete bitsandbytes para funcionar
```bash
pip install accelerete bitsandbytes
```

Para modificar tu script y aplicar cuantificación de 8 bits, puedes hacer lo siguiente:

Este código utiliza cuantificación de 8 bits en lugar de la configuración original de 4 bits, lo que debería resultar en una reducción sustancial en el uso de memoria mientras mantiene un equilibrio adecuado entre eficiencia y precisión.

## Cargar modelo y ejecutar inferencia (mejor cargarlo una vez y luego ejecutarlo varias veces -en la siguiente sección)

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", device_map='auto', load_in_8bit=True, trust_remote_code=True)

#gracias al parámetro device_map='auto' en la función from_pretrained. Este parámetro le indica a la biblioteca Transformers que asigne automáticamente el modelo al mejor dispositivo disponible (normalmente la GPU, si hay una). Por lo tanto, no se necesita llamar explícitamente a .to('cuda')

messages = [
    {'role': 'user', 'content': "write a quick sort algorithm in python."}
]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=32021)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/javier/miniconda3/envs/mistral/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Sure, here is a simple implementation of the Quick Sort algorithm in Python:

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[0]
        less_than_pivot = [x for x in arr[1:] if x <= pivot]
        greater_than_pivot = [x for x in arr[1:] if x > pivot]
        return quick_sort(less_than_pivot) + [pivot] + quick_sort(greater_than_pivot)

# Test the function
print(quick_sort([3, 6, 8, 10, 1, 2, 1]))
```

This code works by selecting a 'pivot' element from the array and partitioning the other elements into two sub-arrays, according to whether they are less than or greater than the pivot. The pivot element is then in its final position. The process is then recursively applied to the sub-arrays.
<|EOT|>


## Cargar modelo

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Inicializa la variable model como None al inicio
model = None
tokenizer = None


# Función para cargar el modelo si aún no está cargado
def load_model():
    global model
    global tokenizer
    if model is None or not hasattr(model, 'num_parameters'):  # Verifica si model está vacío o no parece ser un modelo válido
        print("Cargando modelo...")       
        tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", device_map='auto', load_in_8bit=True, trust_remote_code=True)
        print("Modelo cargado.")
    else:
        print("Modelo ya estaba cargado.")

load_model()

## Inferencia Modelo con gestión de chat

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

def ajustar_contexto(texto, max_longitud=2000, secuencia="### Instruction"):
    # Comprobar si la longitud del texto es mayor que el máximo permitido
    if len(texto) > max_longitud:
        # Buscar la secuencia de ajuste
        indice_secuencia = texto.find(secuencia)

        # Si la secuencia existe
        if indice_secuencia != -1:
            # Retornar el texto desde la secuencia en adelante
            return texto[indice_secuencia:]
        else:
            # Si la secuencia no se encuentra, recortar simplemente a la longitud máxima
            return texto[:max_longitud]
    else:
        return texto

# Ejemplo de uso de la función
# texto_engordado = "Este es el texto adicional que podría hacer que el texto sobrepase los 30 caracteres. ### Instruction Continuación del texto..."
# texto_ajustado = ajustar_contexto(texto_engordado, 30)
# print(texto_ajustado)

def generate_long_chat(historico, input_text, max_additional_tokens=2000, stop=["<|EOT|>"]):


    prompt = f"### Instruction:\n{input_text}\n### Response:\n"
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True) # para streamear el output pero sin repetir el prompt ni el contexto anterior. 

    final_prompt = historico + "\n" + prompt
    longitud_prompt_tokens = len(tokenizer.encode(final_prompt))

    inputs = tokenizer(final_prompt, return_tensors="pt", add_special_tokens=False)

    model_inputs = inputs.to(model.device)      # .to("cuda")
    outputs = model.generate(**model_inputs,
                             streamer=streamer,
                             max_new_tokens=max_additional_tokens,
                            #  max_length=max_length,
                             temperature=0.1,
                             pad_token_id = 3200,
                             eos_token_id=32021,
                             do_sample=True)

    # Decodificar el tensor de salida a una cadena de texto
    inicio_generado = longitud_prompt_tokens - 1
    decoded_output = tokenizer.decode(outputs[0][inicio_generado:], skip_special_tokens=True)  
    # decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)    
    
    text = final_prompt + decoded_output + "<|EOT|>"
    return text


# para que pueda tener suficiente memoria si queremos algo más de contexto.
# tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", device_map='auto', load_in_8bit=True, trust_remote_code=True)

# modelo sin cuantizar (se queda sin memoria con contexto grande)
# tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", torch_dtype="auto", trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", device_map='auto', torch_dtype="auto", trust_remote_code=True)
#torch_dtype=torch.float16  (half precision) or torch.float32 (single precision que sería absurdo porque deepseek coder viene de llama 2 cuyo parámetros son float16)

load_model()

system_prompt = """
You are an expert AI programming assistant, utilizing the DeepSeek Coder model, and you only answer questions related to computer science.
"""

import sys
import os


historico = system_prompt

while True:
    # read input
    input_text = input("user: ")
    if input_text == "/exit": break
    if input_text == "/historico": 
        print(historico)
        continue
    if input_text == "/len": 
        print("longitud del contexto en caracteres: ", len(historico))
        continue
    if input_text == "/clear":
        historico = ""
        continue
    # generate response
    historico = generate_long_chat(historico, input_text=input_text, max_additional_tokens=2048)
    historico = ajustar_contexto(historico)
    # print response
    # print(salida)
    print(f"\n################################################\n")

Modelo ya estaba cargado.
El problema de las torres de Hanoi es un problema de recursividad. Consiste en mover un n�mero n de discos de un poste a otro, con la restricción de que un disco más grande no puede estar por encima de un disco más pequeño.

Aquí está el algoritmo para resolver el problema de las torres de Hanoi:

1. Mover n-1 discos de la torre origen a la torre auxiliar, usando la torre destino.
2. Mover el disco restante de la torre origen a la torre destino.
3. Mover los n-1 discos de la torre auxiliar a la torre destino usando la torre origen.

Aquí está el pseudocódigo para el problema de las torres de Hanoi:

```
procedure hanoi(n, source, target, auxiliary)
   if n > 0 then begin
      // Mover los n-1 discos de la torre source a la torre auxiliary, usando target como torre auxiliar
      hanoi(n - 1, source, auxiliary, target);
      
      // Mover el disco restante de la torre source a la torre target
      move disk from source to target;
      
      // Mover los 

# Usando Transformers con cuantificación de 4 bits (BitsandBytes)

La librería Bits and Bytes (BitsAndBytes) es una herramienta que facilita la optimización de modelos de lenguaje grandes (como GPT-3 y similares) para su ejecución en hardware con recursos limitados. Estas optimizaciones incluyen técnicas como la cuantificación a 4 bits y el uso de tipos de datos de menor precisión para reducir el uso de memoria y mejorar la eficiencia de la inferencia. 

Veamos en detalle los parámetros que mencionaste en tu configuración de `BitsAndBytesConfig`:

1. **`load_in_4bit`**: 
   - Este parámetro, cuando se establece en `True`, habilita la cuantificación a 4 bits. La cuantificación es una técnica para reducir la precisión de los números usados en un modelo (por ejemplo, de 32 bits a 4 bits). Esto disminuye el tamaño del modelo y puede reducir el uso de memoria, pero potencialmente a costa de alguna pérdida de precisión en los cálculos.

2. **`bnb_4bit_quant_type`**:
   - Define el tipo de cuantificación a 4 bits a utilizar. El valor `'nf4'` se refiere a una forma específica de cuantificación a 4 bits desarrollada por el equipo de BitsAndBytes. Esta forma de cuantificación está diseñada para mantener un equilibrio entre el rendimiento y la precisión.

3. **`bnb_4bit_use_double_quant`**:
   - Cuando se establece en `True`, activa el uso de "doble cuantificación" en el proceso de cuantificación a 4 bits. Esto significa que se aplican dos rondas de cuantificación en lugar de una, lo que puede mejorar la precisión de la representación de 4 bits a costa de una ligera sobrecarga computacional.

4. **`bnb_4bit_compute_dtype`**:
   - Este parámetro especifica el tipo de dato a utilizar para los cálculos internos del modelo cuando se utiliza la cuantificación a 4 bits. El tipo de dato `bfloat16` es una representación de punto flotante de 16 bits con menos precisión que el estándar de 32 bits (`float32`), pero con un rango comparable. Usar `bfloat16` puede ofrecer un buen equilibrio entre el rendimiento y la precisión, y es especialmente útil en hardware que soporta operaciones de `bfloat16` de manera eficiente.

En resumen, estos parámetros están configurando tu modelo para que use cuantificación a 4 bits con un método específico y una doble cuantificación para mejorar la precisión, todo mientras realiza cálculos internos en un tipo de dato de menor precisión (`bfloat16`) para mejorar la eficiencia. Estas optimizaciones están diseñadas para permitir que modelos grandes se ejecuten en hardware con memoria limitada, como GPUs con menos VRAM, manteniendo un buen equilibrio entre el rendimiento y la precisión.

In [23]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from torch import bfloat16

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)


tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", quantization_config=bnb_config,     device_map='auto', trust_remote_code=True)
messages=[
    { 'role': 'user', 'content': "write a quick sort algorithm in python."}
]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
# 32021 is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=32021)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/javier/miniconda3/envs/mistral/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Sure, here is a simple implementation of the Quick Sort algorithm in Python:

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[len(arr) // 2]
        left = [x for x in arr if x < pivot]
        middle = [x for x in arr if x == pivot]
        right = [x for x in arr if x > pivot]
        return quick_sort(left) + middle + quick_sort(right)

# Test the function
print(quick_sort([3,6,8,10,1,2,1]))
```

This function works by selecting a pivot element from the array and partitioning the other elements into two sub-arrays, according to whether they are less than or greater than the pivot. The sub-arrays are then recursively sorted.



# Usando Transformers con cuantificación de 16 bits (convirtiendo a half-precision)

Al usar parámetro torch_dtype="auto"  en la función from_pretrained() de la biblioteca Transformers de Hugging Face es una funcionalidad relativamente reciente que permite a la función determinar automáticamente el tipo de datos de PyTorch (dtype) más adecuado para cargar el modelo, basándose en el hardware disponible.

Cuando configuras torch_dtype en "auto", la biblioteca intentará elegir el mejor tipo de datos para optimizar el rendimiento y la eficiencia de memoria. Por ejemplo, si se detecta una GPU compatible con half-precision (como las GPUs con soporte para Tensor Cores de NVIDIA), puede cargar automáticamente el modelo en "float16" para aprovechar la aceleración de half-precision. Si no se detecta dicho hardware, utilizará el tipo de datos estándar (usualmente float32).

Automáticamente se carga usando half-precision (16 bits) cuando se usa la biblioteca Accelerate. Esto se debe a que Accelerate utiliza la biblioteca NVIDIA Apex para la conversión automática a half-precision. Puedes ver esto en la documentación de Accelerate

In [1]:
%pip show accelerate

Name: accelerate
Version: 0.24.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /home/javier/miniconda3/envs/mistral/lib/python3.10/site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, torch
Required-by: peft
Note: you may need to restart the kernel to use updated packages.


En la biblioteca Transformers de Hugging Face, los valores para `device_map` y `torch_dtype` se pueden definir de la siguiente manera:

1. **`device_map`**:
   - **`'auto'`**: El modelo se distribuye automáticamente a través de los dispositivos disponibles de manera óptima. Esto es útil si tienes múltiples GPUs y deseas que la biblioteca maneje la distribución del modelo de manera eficiente.
   - **Diccionario específico de dispositivos**: Puedes proporcionar un diccionario que asigne capas específicas del modelo a dispositivos específicos. Por ejemplo, `{0: [0, 1], 1: [2, 3]}` asignaría las capas 0 y 1 a la GPU 0 y las capas 2 y 3 a la GPU 1.
   - **Lista de IDs de dispositivos**: También puedes proporcionar una lista de IDs de dispositivos para un reparto más manual.

2. **`torch_dtype`**:
   - **`'auto'`**: Selecciona automáticamente el tipo de datos de PyTorch más adecuado en función del hardware disponible y el modelo específico.
   - **Tipos de datos específicos de PyTorch**: Puedes especificar un tipo de datos concreto como `torch.float32`, `torch.float16`, etc., para controlar la precisión y el uso de memoria del modelo.

Estos parámetros proporcionan flexibilidad en la configuración del hardware y el rendimiento del modelo, permitiendo optimizaciones según el entorno de ejecución y los requisitos específicos.

Aquí en un modelo de 32 capas típico Llama asignamos la mitad de las capas a la GPU y la otra mitad a la CPU:
```python
device_map = {
    "cuda:0": [0, 1, 2, ..., 15],  # Primeras 16 capas en la GPU
    "cpu": [16, 17, 18, ..., 31]   # Últimas 16 capas en la CPU
}

model = LlamaForCausalLM.from_pretrained("tu-modelo", device_map=device_map)

## usamos device_map='auto' para que la biblioteca maneje la distribución del modelo de manera eficiente.
Esta manera es muy sencilla, intenta poner el modelo en el mejor dispositivo PERO si no cupiera todo el modelo en la GPU daría un error. Con este parámetro a "auto" se asigna un disposivivo que se puede verse con "model.device" 

Otra forma de gestionar sería sin usar el parámetro device_map, entonces todo se cargaría en la RAM y a partir de ahí se podría usar instrucciones como:
```python
model = model.to(dtype=torch.float16, device='cuda' if torch.cuda.is_available() else 'cpu')
```
para asignar el tipo de dato y el dispositivo donde volcar el modelo.
Esta forma postergaría tanto la cuantificación a half_precision como la carga a la GPU hasta esta linea.

Si no tenemos memoria suficiente en la RAM se podría usar "dtype=torch.float16" O "dtype='auto'" durante la carga del modelo y luego usar
```python
model = model.to(device='cuda' if torch.cuda.is_available() else 'cpu')
```
para trasladar el modelo a la GPU.

## Mínima cuantificación para mayor precisión: "torch_dtype=torch.float16" o "auto"

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", torch_dtype="auto", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", device_map='auto', torch_dtype="auto", trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
messages=[
    { 'role': 'user', 'content': "write a quick sort algorithm in python."}
]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
# 32021 is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=32021)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

/home/javier/miniconda3/envs/mistral/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Sure, here is a simple implementation of the Quick Sort algorithm in Python:

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[0]
        less_than_pivot = [x for x in arr[1:] if x <= pivot]
        greater_than_pivot = [x for x in arr[1:] if x > pivot]
        return quick_sort(less_than_pivot) + [pivot] + quick_sort(greater_than_pivot)

# Test the function
arr = [10, 7, 8, 9, 1, 5]
print("Original array:", arr)
print("Sorted array:", quick_sort(arr))
```

This code works by selecting a 'pivot' element from the array and partitioning the other elements into two sub-arrays, according to whether they are less than or greater than the pivot. The pivot element is then in its final position. The process is then repeated for the sub-arrays.
<|EOT|>


## Para ver el formato que usa: https://github.com/deepseek-ai/deepseek-coder

```bash
You are an AI programming assistant, utilizing the DeepSeek Coder model, developed by DeepSeek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer.
### Instruction:
['content']
### Response:
['content']
<|EOT|>
### Instruction:
['content']
### Response:

In [9]:
messages = [
    {'role': 'user', 'content': "write a quick sort algorithm in python."}
]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

# Decodificar el tensor completo a texto
text = tokenizer.decode(inputs.flatten(), skip_special_tokens=True)
print(text)

You are an AI programming assistant, utilizing the Deepseek Coder model, developed by Deepseek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer
### Instruction:
write a quick sort algorithm in python.
### Response:



# Ver capas del modelo

In [9]:
from transformers import AutoModel

def count_model_layers(model):
    #En nuestro caso entrará en este primer if
    if hasattr(model, 'model') and hasattr(model.model, 'layers'):
        return len(model.model.layers)
    elif hasattr(model, 'encoder'):
        return len(model.encoder.layer)
    elif hasattr(model, 'transformer'):
        return len(model.transformer.h)
    elif hasattr(model, 'decoder'):
        return len(model.decoder.block)
    else:
        return "No se puede determinar el número de capas"


print(f"El modelo tiene {count_model_layers(model)} capas.")


El modelo tiene 32 capas.


In [8]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaLinearScalingRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm

## Usamos device_map "a medida" para intentar que el modelo "quepa" en la GPU

No he logrado que funcionen para este modelo, dan error, solo funcionan con "auto" (investigar más)

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

device_map = {
    "cuda:0": [0, 1, 2, ..., 15],  # Primeras 16 capas en la GPU
    "cpu": [16, 17, 18, ..., 31]   # Últimas 16 capas en la CPU
}
device_map = {
    "cuda:0": [0, 1, 2, ..., 15],  # Primeras 16 capas en la GPU
    "cpu": "default"   # resto de capas en la CPU
}

device_map = {"cuda:0": "default"}

#SOLO CONSIGO QUE FUNCIONE "DEVICE_MAP" CON AUTO (TENGO QUE INVESTIGAR MÁS)

device_map = "auto"

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", torch_dtype="auto", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", device_map=device_map, torch_dtype="auto", trust_remote_code=True)


# model = LlamaForCausalLM.from_pretrained("tu-modelo", device_map=device_map)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Otra forma de hacerlo es convertirlo a half-precision después de cargarlo pero necesitas tener más memoria RAM disponible.
Teniendo la mitad de VRAM (GPU) que de RAM se podría cargar un modelo grande y después convertirlo a half-precision (16 bits) para poder ejecutarlo en la GPU.

Para modificar tu script para usar cuantificación de 16 bits (también conocida como precisión mixta o half-precision), puedes convertir el modelo a float16 después de cargarlo. Esto se puede hacer utilizando el método .to() disponible en modelos de PyTorch. Aquí está el código ajustado:

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True)

# Convertir el modelo a float16 y transferirlo a la GPU si está disponible
model = model.to(dtype=torch.float16, device='cuda' if torch.cuda.is_available() else 'cpu')

messages = [
    {'role': 'user', 'content': "write a quick sort algorithm in python."}
]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=32021)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))


# Inferencia de DeepSeek Coder

In [4]:
messages = [
    {'role': 'user', 'content': "write a quick sort algorithm in python."}
]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
outputs = model.generate(inputs, max_new_tokens=512, do_sample=True, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=32021)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Here is a Python implementation of the quick sort algorithm:

```python
def quickSort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quickSort(left) + middle + quickSort(right)

print(quickSort([3,6,8,10,1,2,1]))
# Output: [1, 1, 2, 3, 6, 8, 10]
```

The quick sort algorithm works by selecting a 'pivot' element from the array and partitioning the other elements into two sub-arrays, according to whether they are less than or greater than the pivot. The sub-arrays are then recursively sorted.

This implementation is a bit more complex than necessary for a quick sort, because it creates additional lists. A more space-efficient version of quick sort might involve swapping elements in-place, although this can be more complex to implement.



## uso de streaming

In [2]:
from transformers import TextStreamer
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True) # para streamear el output pero sin repetir el prompt ni el contexto anterior.

content = """
write a quick sort algorithm in python.
"""
messages = [
    {'role': 'user', 'content': content}
]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
outputs = model.generate(inputs, max_new_tokens=512, streamer=streamer, do_sample=True, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=32021)
# print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Here's a basic implementation of Quick Sort algorithm in Python:

```python
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[0]
        less_than_pivot = [x for x in arr[1:] if x <= pivot]
        greater_than_pivot = [x for x in arr[1:] if x > pivot]
        return quicksort(less_than_pivot) + [pivot] + quicksort(greater_than_pivot)

# Test the function
arr = [3, 6, 8, 10, 1, 2, 1]
print("Original Array: ", arr)
print("Sorted Array: ", quicksort(arr))
```

In this implementation, the `quicksort` function takes in a list `arr` and:

- If the list is empty or contains only one element, it is already sorted, so it just returns the list.
- It selects the first element of the list as the pivot and creates two lists: `less_than_pivot` and `greater_than_pivot`.
- It then recursively sorts the `less_than_pivot` and `greater_than_pivot` lists, and combines the three lists to form the final sorted array.



## Varias secuencias de salida

In [20]:
from transformers import TextStreamer

content = """
write a quick sort algorithm in python.
"""
messages = [
    {'role': 'user', 'content': content}
]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
outputs = model.generate(inputs, max_new_tokens=512, do_sample=True, top_k=50, top_p=0.95, num_return_sequences=2, eos_token_id=32021)

print("Secuencia 1:")
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

print("\nSecuencia 2:")
print(tokenizer.decode(outputs[1][len(inputs[0]):], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Secuencia 1:
Here is a quick sort algorithm implemented in Python:

```python
def quickSort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[len(arr) // 2]
        left = [x for x in arr if x < pivot]
        middle = [x for x in arr if x == pivot]
        right = [x for x in arr if x > pivot]
        return quickSort(left) + middle + quickSort(right)

# Testing the function
print(quickSort([3,6,8,10,1,2,1]))
# Output: [1, 1, 2, 3, 6, 8, 10]
```

The quickSort function works by selecting a 'pivot' element from the array and partitioning the other elements into two sub-arrays, according to whether they are less than or greater than the pivot. The sub-arrays are then recursively sorted.


Secuencia 2:
Sure, here is a Python implementation of the Quick Sort algorithm:

```python
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    righ

## Busqueda de la mejor secuencia de salida (Beam Search) [tarda bastante más pero puede ser más precisa]

In [21]:
from transformers import TextStreamer

content = """
write a quick sort algorithm in python.
"""
messages = [
    {'role': 'user', 'content': content}
]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
outputs = model.generate(inputs, max_new_tokens=512, do_sample=True, top_k=50, top_p=0.95, num_beams=4, num_return_sequences=1, eos_token_id=32021)

print("Secuencia 1:")
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Secuencia 1:
Sure, here is a simple implementation of the Quick Sort algorithm in Python:

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[0]
        less_than_pivot = [x for x in arr[1:] if x <= pivot]
        greater_than_pivot = [x for x in arr[1:] if x > pivot]
        return quick_sort(less_than_pivot) + [pivot] + quick_sort(greater_than_pivot)

# Test the function
print(quick_sort([3, 6, 8, 10, 1, 2, 1]))  # Output: [1, 1, 2, 3, 6, 8, 10]
```

This algorithm works by selecting a 'pivot' element from the array and partitioning the other elements into two sub-arrays, according to whether they are less than or greater than the pivot. The sub-arrays are then recursively sorted.



# Usando llama-cpp-python (sin uso de Docker) (NO FUNCIONA)

## Usando llama.cpp

In [ ]:
%%bash
# echo `pwd`
cd ./llama.cpp/examples/
./chat.sh

In [1]:
from llama_cpp import Llama
llm = Llama(model_path="./models/deepseek-coder-6.7b-instruct.Q6_K.gguf", device_map="auto")
output = llm(
    "Q: Name the planets in the solar system? A: ", # Prompt
    max_tokens=32, # Generate up to 32 tokens
    stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
    echo=True # Echo the prompt back in the output
) # Generate a completion, can also call create_completion
print(output)
# {
#   "id": "cmpl-xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx",
#   "object": "text_completion",
#   "created": 1679561337,
#   "model": "./models/7B/llama-model.gguf",
#   "choices": [
#     {
#       "text": "Q: Name the planets in the solar system? A: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune and Pluto.",
#       "index": 0,
#       "logprobs": None,
#       "finish_reason": "stop"
#     }
#   ],
#   "usage": {
#     "prompt_tokens": 14,
#     "completion_tokens": 28,
#     "total_tokens": 42
#   }
# }

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4080, compute capability 8.9
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ./models/deepseek-coder-6.7b-instruct.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q6_K     [  4096, 32256,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q6_K     [  4096, 11008,     1,     1 ]
llama_

In [4]:
output = llm(
    "Q: Name the planets in the solar system? A: ", # Prompt
    max_tokens=512, # Generate up to 32 tokens
    # stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
    echo=True # Echo the prompt back in the output
) # Generate a completion, can also call create_completion
print(output)

Llama.generate: prefix-match hit


{'id': 'cmpl-82d11f81-51ab-4402-97ce-cc85c3a4ebb5', 'object': 'text_completion', 'created': 1701680303, 'model': './models/deepseek-coder-6.7b-instruct.Q6_K.gguf', 'choices': [{'text': 'Q: Name the planets in the solar system? A: \nTerra, Saturno, Urano e Netuno.\nB: Mercrio, Venus, Marte, Jpiter, Saturno, Urano e Netuno.\nC: Terra, Vênus, Marte, Jpiter, Saturno, Urano, Netuno e Plutão.\nD: Mercurio, Vénus, Mars, Jupiter, Saturn, Uranus and Neptune.\nE: Mercrio, Venus, Terra, Marte, Jpiter, Saturno, Urano e Netuno.\nF: Não é uma alternativa válida pois o planeta Plutão não está na lista. \nG: Está errada pois a letra G não faz parte dos planetas do sistema solar. \nAcertou: Falso, mas não sabemos qual foi a resposta correta.\n\nQuem pode ajudar?\nOBS: Esta pergunta pertence ao grupo de discusses no Quora (https://www.quora.com/q/topic/Planet-Names) e está sendo migranda para o StackExchange em 2016.\n\nA: Terra, Vênus, Marte, Jpiter, Saturno, Urano e Netuno.\n\nEsses são os nomes de to


llama_print_timings:        load time =     820.72 ms
llama_print_timings:      sample time =      68.68 ms /   498 runs   (    0.14 ms per token,  7250.81 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   43896.59 ms /   498 runs   (   88.15 ms per token,    11.34 tokens per second)
llama_print_timings:       total time =   44593.99 ms


# Usando llama-cpp-python (No consigo usar la GPU, uso Docker)

```bash
docker run  -it -p 8000:8000 --name deepseek -v /mnt/d/DeveloperIA/DeepSeekCoder/models:/models -e MODEL=/models/deepseek-coder-6.7b-instruct.Q6_K.gguf ghcr.io/abetlen/llama-cpp-python:latest

## entrar dentro del docker con bash
docker exec -it deepseek bash

## EJecutar inferencia mediante api

In [1]:
import requests

prompt = "write a quick sort algorithm in python." 

response = requests.post("http://localhost:8000/v1/completions", json={
    "prompt": prompt,
    "max_tokens": 512,
    "eos_token_id": 32021,
    #uso gpu
    "device": "cuda:0"
})

print(response.json()["choices"][0]["text"])
# print(response.json())


#include<iostream>
using namespace std;
int partition(int A[], int start, int end) {
    int pivot = A[end]; // taking last element as the pivot 
    int pIndex= start; // pointer for greater elements
    for (int i = start; i < end; i++){ // traverse through all array elements 
        if(A[i] <= pivot) {// if current item is smaller than the pivot, swap it with the greater element pointed by pIndex
            swap(A[i], A[pIndex]); 
            ++pIndex;
        }
    }
    // swapping pivot to its final position
    swap (A[end], A[pIndex]);
    return pIndex;
}
void QuickSort(int A[], int start, int end) {
    if(start < end){ 
        int pIndex = partition(A, start, end); // call the partition function to get the pivot index  
        QuickSort(A, start , pIndex -1 ); // recursively call for elements on left of pivot
        QuickSort(A, pIndex + 1, end);  // recursively call for elements on right side of pivot
    }
}
int main(){
    int A[] = {7,2,1,6,8,5,3,4};// initialize a

In [5]:
%%writefile check_cuda.cu
#include <cuda_runtime.h>
#include <stdio.h>

int main() {
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    if (deviceCount > 0) {
        printf("GPU is available\n");
    } else {
        printf("GPU is not available\n");
    }
    return 0;
}



Writing check_cuda.cu


In [6]:
!nvcc check_cuda.cu -o check_cuda

In [7]:
!./check_cuda

GPU is available


# Verificar si tengo funcionando cuBLAS (aceleración de GPU)


In [1]:
%%writefile check_cublas.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>

int main() {
    cublasHandle_t handle;
    cublasStatus_t status;

    // Inicializar CUBLAS
    status = cublasCreate(&handle);
    if (status != CUBLAS_STATUS_SUCCESS) {
        printf("CUBLAS no está instalado o no funciona correctamente.\n");
        return -1;
    }

    // Ejemplo sencillo: Escalar un vector
    const int n = 10;
    float alpha = 2.0f;
    float x[n] = {1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0};
    float *d_x;

    // Asignar memoria en la GPU
    cudaMalloc((void **)&d_x, n * sizeof(float));

    // Copiar datos al dispositivo
    cublasSetVector(n, sizeof(float), x, 1, d_x, 1);

    // Escalar el vector x por alpha y almacenar el resultado en x
    cublasSscal(handle, n, &alpha, d_x, 1);

    // Copiar los resultados de vuelta a la memoria del host
    cublasGetVector(n, sizeof(float), d_x, 1, x, 1);

    // Limpiar
    cudaFree(d_x);
    cublasDestroy(handle);

    printf("CUBLAS está instalado y funcionando correctamente.\n");
    return 0;
}


Writing check_cublas.cu


Este script en C realiza los siguientes pasos:

1. Inicializa cuBLAS: Intenta crear un handle de cuBLAS. Si falla, significa que cuBLAS no está disponible.

1. Define y Escala un Vector: Define un vector simple y lo escala usando la función cublasSscal.

1. Manejo de Memoria: Asigna memoria en la GPU, copia los datos del host al dispositivo, realiza la operación y luego copia los datos de vuelta.

1. Limpieza: Libera la memoria y destruye el handle de cuBLAS.



In [3]:
!nvcc -lcublas -o cublas_test check_cublas.cu

In [4]:
!./cublas_test

CUBLAS está instalado y funcionando correctamente.


# Uso de Pytorch o Tensorflow junto con Hugging Face Transformers

La biblioteca transformers de Hugging Face está diseñada para ser compatible tanto con PyTorch como con TensorFlow. Al usar transformers, puedes elegir trabajar con cualquiera de estos frameworks de aprendizaje profundo, dependiendo de tu preferencia y necesidades específicas. Aquí te explico cómo funciona esto:

Compatibilidad con PyTorch y TensorFlow:

Cuando utilizas transformers, puedes especificar si deseas que los tensores sean compatibles con PyTorch o TensorFlow mediante el parámetro return_tensors. Por ejemplo, return_tensors="pt" te dará tensores compatibles con PyTorch, mientras que return_tensors="tf" te dará tensores compatibles con TensorFlow.
Muchos modelos en transformers están disponibles tanto en versiones de PyTorch (AutoModelFor...) como en versiones de TensorFlow (TFAutoModelFor...).
Trabajo de Bajo Nivel:

Si tu trabajo requiere manipulaciones de bajo nivel de tensores o un control más fino sobre el entrenamiento del modelo, tu elección entre PyTorch y TensorFlow puede depender de tu familiaridad con estos frameworks y sus respectivas características.
PyTorch es conocido por su facilidad de uso, especialmente para la investigación y el desarrollo de prototipos, gracias a su naturaleza imperativa y a la forma en que maneja la diferenciación automática.
TensorFlow, especialmente en su versión 2 y con la interfaz Keras, también es muy accesible y ofrece un fuerte soporte para la producción y el escalado gracias a su modelo de gráficos computacionales y herramientas como TensorFlow Serving.

# Comprobar resultados

In [ ]:
def partition(arr, low, high):
    i = (low-1)
    pivot = arr[high]

    for j in range(low, high):
        if arr[j] <= pivot:
            i = i+1
            arr[i], arr[j] = arr[j], arr[i]

    arr[i+1], arr[high] = arr[high], arr[i+1]
    return (i+1)

def quickSort(arr, low, high):
    if len(arr) == 1:
        return arr
    if low < high:
        pi = partition(arr, low, high)
        quickSort(arr, low, pi-1)
        quickSort(arr, pi+1, high)

arr = [10, 7, 8, 9, 1, 5]
n = len(arr)
quickSort(arr, 0, n-1)
print("Sorted array is:", arr)


In [ ]:
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[len(arr) // 2]
        left = [x for x in arr if x < pivot]
        middle = [x for x in arr if x == pivot]
        right = [x for x in arr if x > pivot]
        return quick_sort(left) + middle + quick_sort(right)

# Example usage:
arr = [3, 6, 8, 10, 1, 2, 1]
print(quick_sort(arr))  # prints "[1, 1, 2, 3, 6, 8, 10]"

# Tunel y proxy Socks

In [1]:
import importlib
import sys
# he creado un modulo de herramientas con los enlaces simbólicos a los modulos que uso en los notebooks
# insertar carpeta si no esta en sys.path
if not '../herramientas/' in sys.path:
    print("importando herramientas")
    sys.path.insert(0, '../herramientas/')
import tunel_ssh_socks

importlib.reload(tunel_ssh_socks)

# ver todas las carpetas de sys.path y su orden de busqueda
for path in sys.path:
    print(path)

importando herramientas
../herramientas/
/mnt/d/DeveloperIA/DeepSeekCoder
/home/javier/miniconda3/envs/mistral/lib/python310.zip
/home/javier/miniconda3/envs/mistral/lib/python3.10
/home/javier/miniconda3/envs/mistral/lib/python3.10/lib-dynload

/home/javier/miniconda3/envs/mistral/lib/python3.10/site-packages


In [4]:
import tunel_ssh_socks as tunel

# listar todas las funciones del modulo
# dir(tunel)
tunel.obtener_informacion_ip()

'Dirección IP: 83.138.42.107\nPaís: ES\nRegión: Valencia\nCiudad: Alicante'

# No funciona

In [ ]:
%pip install ctransformers[cuda]

In [1]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained("TheBloke/deepseek-coder-6.7B-instruct-GGUF", model_file="deepseek-coder-6.7b-instruct.Q6_K.gguf", model_type="deepseek", gpu_layers=50)

print(llm("AI is going to"))

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

: 

In [1]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate(template=template, input_variables=["question"])

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/mnt/d/DeveloperIA/DeepSeekCoder/deepseek-coder-6.7b-instruct.Q6_K.gguf",
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    # callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

In [ ]:
prompt = """
write a quick sort algorithm in python.
"""
llm(prompt)